# Set up individual pairwise comparisons

In [1]:
from itertools import combinations
from pathlib import Path
import pandas as pd
import glob
import nibabel as nib
from randomise_prep import setup_randomise_tfce

from survey_medley_code.config_loader import load_config

In [2]:
cfg = load_config()

In [3]:
sub_id = []
bold_paths = []
questionnaire = []

questionnaire_names = ['grit', 'brief', 'future_time', 'impulsive_venture', 'upps']

question_output_path = (cfg.output_root / 'within_subject_brain_behavior_by_questionnaire/within_subject_results')

outlier_file = pd.read_csv(cfg.output_root / 'within_subject_brain_behavior_by_questionnaire/behavioral_measures_effect_size_outlier_analysis/percent_outlier_data.csv')
good_subs_df = outlier_file[outlier_file['image_outlier_percentage'] <= 8]
good_subs_list = sorted(set(good_subs_df['subject_label'].tolist()))

for questionnaire_name in questionnaire_names:
    for good_sub in good_subs_list:
        sub_label = str(good_sub).zfill(3)
        # Glob for matching files
        bold_paths_loop = list(
            question_output_path.glob(
                f'{questionnaire_name}_behavioral_measures_effect_size_sub_{sub_label}.nii.gz'
            )
        )
        if bold_paths_loop:
            sub_id.append(sub_label)
            questionnaire.append(questionnaire_name)
            bold_paths.append(bold_paths_loop[0])  # take the first match
        else:
            print(
                f'Output file missing for question/subject: {questionnaire_name}/{good_sub}'
            )

Output file missing for question/subject: impulsive_venture/373
Output file missing for question/subject: impulsive_venture/513
Output file missing for question/subject: impulsive_venture/541
Output file missing for question/subject: impulsive_venture/553
Output file missing for question/subject: impulsive_venture/558
Output file missing for question/subject: impulsive_venture/561
Output file missing for question/subject: impulsive_venture/577
Output file missing for question/subject: impulsive_venture/581
Output file missing for question/subject: impulsive_venture/585
Output file missing for question/subject: impulsive_venture/588
Output file missing for question/subject: impulsive_venture/591
Output file missing for question/subject: impulsive_venture/596
Output file missing for question/subject: impulsive_venture/601
Output file missing for question/subject: impulsive_venture/602
Output file missing for question/subject: impulsive_venture/605
Output file missing for question/subject

In [10]:
# Directory for outputs
base_out = Path(
    '/oak/stanford/groups/russpold/data/uh2/aim1/derivatives/'
    'survey_medley_results/within_subject_brain_behavior_by_questionnaire/'
    'all_paired_t_tests/paired_differences'
)

# -----------------------------------------------------
# BUILD A LOOKUP:  (sub, questionnaire) → file path
# -----------------------------------------------------

lookup = {}
for sid, qp, qname in zip(sub_id, bold_paths, questionnaire):
    lookup[(sid, qname)] = qp

# All questionnaire names present
questionnaire_names = sorted(set(questionnaire))

# -----------------------------------------------------
# LOOP OVER ALL QUESTIONNAIRE PAIRS
# -----------------------------------------------------
for qa, qb in combinations(questionnaire_names, 2):
    pairing_name = f'{qa}_minus_{qb}'
    outdir = base_out / pairing_name
    outdir.mkdir(parents=True, exist_ok=True)

    print(f'Processing pairing: {pairing_name}')

    # For every subject, check whether both images exist
    for sid in sub_id:
        keyA = (sid, qa)
        keyB = (sid, qb)

        if keyA not in lookup or keyB not in lookup:
            continue  # skip subjects missing data

        imgA = nib.load(str(lookup[keyA]))
        imgB = nib.load(str(lookup[keyB]))

        diff = imgA.get_fdata() - imgB.get_fdata()

        # Save output file
        out_path = outdir / f'{pairing_name}_{sid}.nii.gz'

        nib.Nifti1Image(diff, imgA.affine, imgA.header).to_filename(str(out_path))

        print(f'Saved {out_path}')

Processing pairing: brief_minus_future_time
Saved /oak/stanford/groups/russpold/data/uh2/aim1/derivatives/survey_medley_results/within_subject_brain_behavior_by_questionnaire/all_paired_t_tests/paired_differences/brief_minus_future_time/brief_minus_future_time_061.nii.gz
Saved /oak/stanford/groups/russpold/data/uh2/aim1/derivatives/survey_medley_results/within_subject_brain_behavior_by_questionnaire/all_paired_t_tests/paired_differences/brief_minus_future_time/brief_minus_future_time_130.nii.gz
Saved /oak/stanford/groups/russpold/data/uh2/aim1/derivatives/survey_medley_results/within_subject_brain_behavior_by_questionnaire/all_paired_t_tests/paired_differences/brief_minus_future_time/brief_minus_future_time_172.nii.gz
Saved /oak/stanford/groups/russpold/data/uh2/aim1/derivatives/survey_medley_results/within_subject_brain_behavior_by_questionnaire/all_paired_t_tests/paired_differences/brief_minus_future_time/brief_minus_future_time_192.nii.gz
Saved /oak/stanford/groups/russpold/data/uh2

In [4]:
mask_path = (cfg.output_root / 'questionnaire_brain_behavior_slope_ftest/group_mask.nii.gz')
base_dir = Path(
    '/oak/stanford/groups/russpold/data/uh2/aim1/derivatives/survey_medley_results/within_subject_brain_behavior_by_questionnaire'
)
paired_diff_root = base_dir / 'all_paired_t_tests' / 'paired_differences'
randomise_root = base_dir / 'all_paired_t_tests'

for q1, q2 in combinations(questionnaire_names, 2):
    # --------------------------------------------
    # 1. Build the pairing name and collect files
    # --------------------------------------------
    pair_name = f'{q1}_minus_{q2}'
    diff_dir = paired_diff_root / pair_name

    bold_paths_loop = sorted(diff_dir.glob('*.nii.gz'))
    if not bold_paths_loop:
        print(f'⚠️ No paired difference images found for {pair_name}, skipping.')
        continue

    print(f'Found {len(bold_paths_loop)} paired images for {pair_name}')

    # --------------------------------------------
    # 2. Build output randomise directory
    # --------------------------------------------
    output_paired_ttest_loop = randomise_root / f'paired_test_{pair_name}'
    output_paired_ttest_loop.mkdir(parents=True, exist_ok=True)

    # --------------------------------------------
    # 3. Run TFCE randomise job
    # --------------------------------------------
    script_path = setup_randomise_tfce(
        input_files=bold_paths_loop,
        group_mask=mask_path,  # already defined earlier
        output_directory=output_paired_ttest_loop,
        analysis_type='onesample_2sided',
        num_perm=5000,
        rename_output=True,
    )

    print(f'✓ Created randomise script for {pair_name}: {script_path}')

# TODO: figure out why no difference images were found for grit_minus_brief or grit_minus_future_time
# looks like they were never created in the cell above, but idk why

⚠️ No paired difference images found for grit_minus_brief, skipping.
⚠️ No paired difference images found for grit_minus_future_time, skipping.
Found 65 paired images for grit_minus_impulsive_venture
Concatenating input files...
Created 4D file: /oak/stanford/groups/russpold/data/uh2/aim1/derivatives/survey_medley_results/within_subject_brain_behavior_by_questionnaire/all_paired_t_tests/paired_test_grit_minus_impulsive_venture/input_data4d.nii.gz
Note: rename_output=True will rename _corrp_ to _1minuspvalue_ in output filenames for t-test analysis
✓ Created randomise script for grit_minus_impulsive_venture: /oak/stanford/groups/russpold/data/uh2/aim1/derivatives/survey_medley_results/within_subject_brain_behavior_by_questionnaire/all_paired_t_tests/paired_test_grit_minus_impulsive_venture/randomise_call.sh
Found 93 paired images for grit_minus_upps
Concatenating input files...
Created 4D file: /oak/stanford/groups/russpold/data/uh2/aim1/derivatives/survey_medley_results/within_subject_